In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time

from symbol import Symbol
from helpers import rename_columns, SafeHDFStore, clean_file
from edelweiss import *
import warnings
warnings.filterwarnings('ignore')

In [17]:
s = Symbol()
symbol_list = s.symbol_list

edelweiss_list = load_initial_data('list')
ratio_data = load_initial_data('ratios')
pnl_annual = load_initial_data('pnl_annual')
balancesheet = load_initial_data('balancesheet')
cashflow = load_initial_data('cashflow')
peers = load_initial_data('peers')

edelweiss_list = edelweiss_list.append(pd.DataFrame(index=symbol_list.index))
edelweiss_list = edelweiss_list.reset_index().drop_duplicates(['symbol']).set_index('symbol')
load_list = edelweiss_list.index[edelweiss_list.edelweiss_url.isnull()]
print(load_list)
len(symbol_list)


Index([], dtype='object', name='symbol')


1484

In [6]:
list_symbol = edelweiss_list.index.unique()
ratio_symbol = ratio_data.symbol.unique()
pnl_symbol = pnl_annual.symbol.unique()
bs_symbol = balancesheet.symbol.unique()
cf_symbol = cashflow.symbol.unique()
peer_symbol = peers.symbol.unique()

In [7]:
ls_rs = np.intersect1d(list_symbol, ratio_symbol, True)
pnl_bs = np.intersect1d(pnl_symbol, bs_symbol)
cf_pr = np.intersect1d(cf_symbol, peer_symbol)
ls_bs = np.intersect1d(ls_rs, pnl_bs)
def intersect(*d):
    sets = iter(map(set, d))
    result =  set(d[0]).intersection(*d)
    return list(result)
com = intersect(list_symbol, ratio_symbol, pnl_symbol, bs_symbol, cf_symbol, peer_symbol)
len(com)

1156

In [31]:
def symbol_lsos(symbol):
    edelweiss_list = load_initial_data('list')
    ratio_data = load_initial_data('ratios')
    pnl_annual = load_initial_data('pnl_annual')
    balancesheet = load_initial_data('balancesheet')
    cashflow = load_initial_data('cashflow')
    peers = load_initial_data('peers')
    edelweiss_list.loc[symbol] = np.nan
    ratio_data = ratio_data.drop(ratio_data.loc[ratio_data.symbol == symbol].index)
    pnl_annual = pnl_annual.drop(pnl_annual.loc[pnl_annual.symbol == symbol].index)
    balancesheet = balancesheet.drop(balancesheet.loc[balancesheet.symbol == symbol].index)
    cashflow = cashflow.drop(cashflow.loc[cashflow.symbol == symbol].index)
    peers = peers.drop(peers.loc[peers.symbol == symbol].index)
    
    
    soup = BeautifulSoup(driver.page_source)
    edelweiss_list.loc[symbol] = create_header_list(soup).rename(symbol)
    edelweiss_list = edelweiss_list[[
        'edelweiss_url', 'company_name', 'sector',
        'pe_ratio', 'div_yield', 'mcap', 'beta', 'bullish',
        'bse_identifier', 'nse_identifier', 'isin_identifier',
        'promoter_holding', 'fii_holding', 'dii_mf_holding', 'others_holding'
    ]]
    with SafeHDFStore('edelweiss.h5') as store:
        store.put('list', value=edelweiss_list)
        store.put('ratios', value=ratio_data)
        store.put('pnl_annual', value=pnl_annual)
        store.put('balancesheet', value=balancesheet)
        store.put('cashflow', value=cashflow)
        store.put('peers', value=peers)
    print('Successful for {0}'.format(symbol))
    
symbol_lsos('bel')

Successful for bel


In [ ]:
load_edelweiss_symbol_list(load_list)

In [ ]:
yearly_click = driver.find_element(By.XPATH, '''//a[@gtmdir-label="Yearly"]''')
hlyearly_click = driver.find_element(By.XPATH, '''//a[@gtmdir-label="Half Yearly"]''')
quarterly_click = driver.find_element(By.XPATH, '''//a[@gtmdir-label="Quarterly"]''')
for i in range(10):
    quarterly_click.click()    
    yearly_click.click()
    hlyearly_click.click()

In [ ]:
print(edelweiss_list.columns)
print(ratio_data.columns)
print(pnl_annual.columns)
print(balancesheet.columns)
print(cashflow.columns)
print(peers.columns)


print(len(edelweiss_list.dropna(how='all')))
print(len(ratio_data.symbol.unique()))
print(len(pnl_annual.symbol.unique()))
# print(len(pnl_semi_annual.symbol.unique()))
# print(len(pnl_quarterly.symbol.unique()))
print(len(balancesheet.symbol.unique()))
print(len(peers.symbol.unique()))

In [ ]:
edelweiss_list = load_initial_data('list')
ratio_data = load_initial_data('ratios')
pnl_annual = load_initial_data('pnl_annual')
balancesheet = load_initial_data('balancesheet')
cashflow = load_initial_data('cashflow')
peers = load_initial_data('peers')

In [37]:
def create_header_list(soup):

    edelweiss_list = pd.Series()
    symbol_info_soup = soup.find('div', {'id': 'market-overview'})
    edelweiss_list['edelweiss_url'] = soup.find('meta', {'name': 'twitter:url'})['content'].strip()
    edelweiss_list['company_name'] = symbol_info_soup.find('span', {'ng-bind': '::vm.compname'}).text.strip()
    edelweiss_list['sector'] = symbol_info_soup.find('a', {'href': '', 'class': 'ng-binding'}).text.strip()
    try:
        edelweiss_list['bullish'] = soup.find('div', {'class': 'bullish'}).find('span').text.strip()
    except:
        pass

    basic_stats = soup.find('div', {'class': 'row border border-top mobile-chart-info'})
    basic_stats_spans = basic_stats.find_all('span', {'class': 'value ng-binding'})
    edelweiss_list['pe_ratio'] = basic_stats_spans[1].text.strip()
    edelweiss_list['div_yield'] = basic_stats_spans[2].text.strip()
    edelweiss_list['mcap'] = basic_stats_spans[3].text.replace(',', '').strip()
    edelweiss_list['beta'] = basic_stats_spans[4].text.strip()

    identifier = soup.find('div', {'id': 'shareholding-pattern'})
    identifier_list = identifier.find('div', {'class': 'share'}).find_all('li')
    edelweiss_list['bse_identifier'] = identifier_list[0].text[5:].strip()
    edelweiss_list['nse_identifier'] = identifier_list[1].text[5:].strip()
    edelweiss_list['isin_identifier'] = identifier_list[2].text[6:].strip()
    try:
        shareholding = soup.find('div', {'id': 'shareholding-pattern'})
        shareholding_span = shareholding.find('div', {'config': 'vm.SHChartData'}).find_all('span')
        edelweiss_list['promoter_holding'] = shareholding_span[0].text[:-1].strip().split()[-1]
        edelweiss_list['fii_holding'] = shareholding_span[1].text[:-1].strip().split()[-1]
        edelweiss_list['dii_mf_holding'] = shareholding_span[2].text[:-1].strip().split()[-1]
        edelweiss_list['others_holding'] = shareholding_span[3].text[:-1].strip().split()[-1]
    except:
        edelweiss_list['promoter_holding'] = 0
        edelweiss_list['fii_holding'] = 0
        edelweiss_list['dii_mf_holding'] = 0
        edelweiss_list['others_holding'] = 0
    return edelweiss_list

In [14]:
driver = webdriver.Firefox()
driver.get('https://www.edelweiss.in/quotes/equity/Andhra-Cements-Ltd-19?q=andhra%20cements%20ltd')

In [18]:
soup = BeautifulSoup(driver.page_source)
edelweiss_list.loc['mcdowell-n'] = create_header_list(soup).rename('mcdowell-n')
edelweiss_list = edelweiss_list[[
    'edelweiss_url', 'company_name', 'sector',
    'pe_ratio', 'div_yield', 'mcap', 'beta', 'bullish',
    'bse_identifier', 'nse_identifier', 'isin_identifier',
    'promoter_holding', 'fii_holding', 'dii_mf_holding', 'others_holding'
]]

In [19]:
edelweiss_list.to_hdf('edelweiss.h5', 'list')

In [ ]:
def click(element, driver=driver):
    driver.execute_script("window.scrollTo(0, 0)")
#     time.sleep(1)
#     element.click()
    
def load_pnl_data(driver):
    standalone_element = driver.find_element(By.XPATH, '//a[@ng-click="vm.FormPNLData(vm.PNLActivePeriod,false)"]')
    consolidated_element = driver.find_element(By.XPATH, '//a[@ng-click="vm.FormPNLData(vm.PNLActivePeriod,true)"]')
 
    yearly_ = driver.find_element(By.XPATH, '''//li[@ng-class="{'active':vm.PNLActivePeriod==='Yearly'}"]''')
    hlyearly_ = driver.find_element(By.XPATH, '''//li[@ng-class="{'active':vm.PNLActivePeriod==='HalfYearly'}"]''')
    quarterly_ = driver.find_element(By.XPATH, '''//li[@ng-class="{'active':vm.PNLActivePeriod==='Quarterly'}"]''')

    yearly_click = driver.find_element(By.XPATH, '''//a[@gtmdir-label="Yearly"]''')
    hlyearly_click = driver.find_element(By.XPATH, '''//a[@gtmdir-label="Half Yearly"]''')
    quarterly_click = driver.find_element(By.XPATH, '''//a[@gtmdir-label="Quarterly"]''')
    quarterly_click.click()
    i = 0
    for period, period_click in zip([yearly_, hlyearly_, quarterly_], [yearly_click, hlyearly_click, quarterly_click]):
        if i == 0 and period.get_attribute('class') == 'active':
            pass
        else:
            period_click.click()
        if standalone_element.get_attribute('class') == '':
            standalone_element.click()
        temp_soup = BeautifulSoup(driver.page_source, 'html.parser')
        soum_soup = temp_soup.find('div', {'id': 'financial-health'}).find('table', {'ng-show':'!vm.PNLConsolidated'})
        standalone = pd.read_html(str(soum_soup), index_col=0, header=0)[0]
        standalone['type'] = 'standalone'
        
        if consolidated_element.get_attribute('class') == 'ng-hide':
            consolidated = pd.DataFrame()
        else:
            consolidated_element.click()

            temp_soup = BeautifulSoup(driver.page_source,'html.parser')
            soum_soup = temp_soup.find('div', {'id': 'financial-health'}).find('table', {'ng-show':'vm.PNLConsolidated'})
            consolidated = pd.read_html(str(soum_soup), index_col=0, header=0)[0]
            consolidated['type'] = 'consolidated'
        if i == 0:
            yearly = standalone.append(consolidated).reset_index().replace('-', np.nan)
        elif i == 1:
            hlyearly = standalone.append(consolidated).reset_index().replace('-', np.nan)
        else:
            quarterly = standalone.append(consolidated).reset_index().replace('-', np.nan)
        i = i + 1
        print(i)
    return yearly.reset_index(drop=True), hlyearly.reset_index(drop=True), quarterly.reset_index(drop=True)

def load_balance_sheet(driver):
    standalone_element = driver.find_element(By.XPATH, '//a[@ng-click="vm.BSConsolidated=false"]')
    consolidated_element = driver.find_element(By.XPATH, '//a[@ng-click="vm.BSConsolidated=true"]')
    
    if standalone_element.get_attribute('class') == '':
            standalone_element.click()
    temp_soup = BeautifulSoup(driver.page_source, 'html.parser')
    balancesheet_soup = temp_soup.find('div', {'ui-view': 'balancesheet'})
    standalone_ = pd.read_html(str(balancesheet_soup), index_col=0, header=0)[0]
    standalone_['type'] = 'standalone'

    if consolidated_element.get_attribute('class') == 'ng-hide':
        consolidated_ = pd.DataFrame()
    else:
        consolidated_element.click()

        temp_soup = BeautifulSoup(driver.page_source, 'html.parser')
        balancesheet_soup = temp_soup.find('div', {'ui-view': 'balancesheet'})
        consolidated_ = pd.read_html(str(balancesheet_soup), index_col=0, header=0)[1]
        consolidated_['type'] = 'consolidated'
        
    balancesheet = standalone_.append(consolidated_).reset_index().replace('-', np.nan)
    return balancesheet.reset_index(drop=True)